# 03과 딥러닝 개념 익히기


## 1.환경준비

### (1) 라이브러리 로딩

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense
from keras.backend import clear_session
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

* 학습곡선 그래프

### (2) 데이터로딩

In [ ]:
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/boston.csv'
data = pd.read_csv(path)
data.head()

|	변수	|	설명	|
|	----	|	----	|
|	medv	|	타운별 집값(중위수)	|
|	crim	|	범죄율	|
|	zn	|	25,000 평방피트를 초과 거주지역 비율	|
|	indus	|	비소매상업지역 면적 비율	|
|	chas	|	찰스강변 위치(범주 : 강변1, 아니면 0)	|
|	nox	|	일산화질소 농도	|
|	rm	|	주택당 방 수	|
|	age	|	1940년 이전에 건축된 주택의 비율	|
|	dis	|	직업센터의 거리	|
|	rad	|	방사형 고속도로까지의 거리	|
|	tax	|	재산세율	|
|	ptratio	|	학생/교사 비율	|
|	lstat	|	인구 중 하위 계층 비율	|


## 2.학습 절차 익히기

### (1) 전처리

In [ ]:
# 학습을 위한 데이터 분할
target = 'medv'
features = ['lstat']
x = data.loc[:, features]
y = data.loc[:, target]
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=.2, random_state = 20)

# 스케일링
scaler = MinMaxScaler()
x_train_s = scaler.fit_transform(x_train)
y_train_s = (y_train - y_train.min())/(y_train.max() - y_train.min())

### (2) 학습절차 시각화 함수

* 함수 생성
    * 아래 함수는 내용을 이해하려고 하지 마시고 그냥 생성해 주세요.
    * 눈으로 보여주기 위해서 만든 것입니다. 사용만 할 겁니다.

In [ ]:
def dl_visualize(ep, lr) :
    clear_session()

    model = Sequential([ Dense(1, input_shape = (1,)) ])

    model.compile(loss='mse', optimizer= Adam(learning_rate = lr))
    mcp = ModelCheckpoint(filepath='/content/{epoch:d}.h5',
                        monitor='val_loss', save_best_only=False, save_weights_only=True)

    history = model.fit(x_train_s, y_train_s, verbose = 0, epochs = ep, callbacks=[mcp]).history

    coef,intercept = [],[]

    for i in range(ep) :
        file = f'/content/{i+1}.h5'
        model.load_weights(file)
        coef.append(np.array(model.weights[0])[0,0])
        intercept.append(np.array(model.weights[1])[0])
    
    plt.figure(figsize = (20,8))
    plt.subplot(1,2,1)
    sns.scatterplot(x=x_train_s.reshape(-1,), y=y_train_s, alpha = .5)
    plt.grid()
    plt.xlabel('lstat')
    for i in range(ep):
        x = np.linspace(0,1,10)
        y = coef[i]*x+ intercept[i]
        plt.plot(x, y, 'r--')
        v = 1.005
        plt.text(v, coef[i]*v+ intercept[i], f'ep:{i+1}', color = 'r')

    plt.subplot(1,2,2)
    plt.plot(range(1, ep+1), history['loss'], label='train_err', marker = '.')

    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid()
    plt.show()

### (3) 학습 절차 눈으로 확인하기

#### 1) 실험1
* epochs를 조절해 봅시다.
    * lr = 0.01로 고정시켜 놓고
    * ep를 5, 10, 15 로 조정하며 실행해 봅시다.


In [ ]:
dl_visualize(ep = 30, lr = 0.01)

#### 2) 실험2
* epochs를 조절해 봅시다.
    * ep = 20으로 고정시켜 놓고
    * lr을 0.001, 0.01, 0.05, 0.1 로 조정하며 실행해 봅시다.


In [ ]:
dl_visualize(ep = 30, lr = 0.01)